In [1]:
import requests
import json
import pandas as pd
from Crypto.Cipher import AES
from base64 import b64encode
from pathlib import Path
from faker import Faker

In [2]:
"""
dev tool调试信息:
 function b(a, b) {
        var c = CryptoJS.enc.Utf8.parse(b)
          , d = CryptoJS.enc.Utf8.parse("0102030405060708")
          , e = CryptoJS.enc.Utf8.parse(a)
          , f = CryptoJS.AES.encrypt(
                e, c, {iv: d, mode: CryptoJS.mode.CBC}
                );
        return f.toString()
    }
function d(d, e, f, g) {
    var h = {}
        , i = a(16);
    return h.encText = b(d, g),
    h.encText = b(h.encText, i),
    h.encSecKey = c(i, e, f),
    h
}

params:
"csrf_token": ""  # 不影响加密
"cursor": "-1"  # 不影响加密
"offset": 0,  # 定位到第几条评论
"orderType": "1"  # 评论排序类型
"pageNo": "1"  # 第几页
"pageSize":40  # 每页显示多少条评论
"rid": "A_PL_0_6883421967"  # 不影响加密
"threadId": "A_PL_0_6883421967"  # 线程ID
"""


'\ndev tool调试信息:\n function b(a, b) {\n        var c = CryptoJS.enc.Utf8.parse(b)\n          , d = CryptoJS.enc.Utf8.parse("0102030405060708")\n          , e = CryptoJS.enc.Utf8.parse(a)\n          , f = CryptoJS.AES.encrypt(\n                e, c, {iv: d, mode: CryptoJS.mode.CBC}\n                );\n        return f.toString()\n    }\nfunction d(d, e, f, g) {\n    var h = {}\n        , i = a(16);\n    return h.encText = b(d, g),\n    h.encText = b(h.encText, i),\n    h.encSecKey = c(i, e, f),\n    h\n}\n\nparams:\n"csrf_token": ""  # 不影响加密\n"cursor": "-1"  # 不影响加密\n"offset": 0,  # 定位到第几条评论\n"orderType": "1"  # 评论排序类型\n"pageNo": "1"  # 第几页\n"pageSize":40  # 每页显示多少条评论\n"rid": "A_PL_0_6883421967"  # 不影响加密\n"threadId": "A_PL_0_6883421967"  # 线程ID\n'

In [3]:
def transfer16(text) -> bytes:
    """补全16位字符串，并转换为字节码

    Parameters
    ----------
    text : str
        输入字符串

    Returns
    -------
    bytes
        补全后的字节码
    """
    pad = 16 - len(text) % 16
    enctxt = text + chr(pad) * pad
    return enctxt.encode("utf-8")


def enc_param(text, key):
    """AES加密，加密过程需将字符串转换为字节码

    Parameters
    ----------
    text : str
        输入字符串
    key : str
        输入密钥

    Returns
    -------
    str
        已加密字符串
    """    
    iv = "0102030405060708".encode("utf-8")
    mode = AES.MODE_CBC
    aes = AES.new(key.encode("utf-8"), mode=mode, iv=iv)
    string = transfer16(text)
    cipher = aes.encrypt(string)
    return str(b64encode(cipher), "utf-8")


def get_param(cipher, key1, key2):
    """获取两次加密后的字符串

    Parameters
    ----------
    cipher : str
        输入字符串
    key1 : str
        第一次加密所用密钥
    key2 : str
        第二次加密所用密钥

    Returns
    -------
    str
        已加密字符串
    """    
    pw = enc_param(cipher, key1)
    pw = enc_param(pw, key2)
    return pw


In [4]:
url = "https://music.163.com/weapi/comment/resource/comments/get?csrf_token="
fake = Faker()
data = {
    "offset": 0,  # 定位到第几条评论
    "orderType": 1,  # 评论排序类型
    "pageNo": 1,  # 页数
    "pageSize": 20,  # 每页显示多少条评论，默认为20
    "threadId": "A_PL_0_6883421967",  # 线程ID
    "total": "false",
}
headers = {
    "user-agent": fake.chrome(),
    "referer": "https://music.163.com/playlist?id=6883421967",
}
var1 = "010001"
var2 = "00e0b509f6259df8642dbc35662901477df22677ec152b5ff68ace615bb7b725152b3ab17a876aea8a5aa76d2e417629ec4ee341f56135fccf695280104e0312ecbda92557c93870114af6c9d05c4f7f0c3685b7a46bee255932575cce10b424d813cfe4875d3e82047b97ddef52741d546b8e289dc6935b3ece0462db0a22b8e7"
var3 = "0CoJUm6Qyw8W8jud"
i = "WzpuV0v2CuoIy36f"
encSecKey = "d9eff4f4d69c20aab714826f2f569b0d76ae58d98f3e4c11245e6e486f1f3cffa16ee32facf1583f7fc6258f2d6a471415802eb898f76be85dfd10c5a74b836ae83a1e592c20709b9371d89857c47d2f268bdedcab483706439fabcb8ea53a118750862341300d351da4383bfb835baa1e4d229729209c8920ab220afdd3f082"


In [5]:
count = 0
filename = Path("./comment.txt")
filename.unlink(missing_ok=True)
file = filename.open("a", encoding='utf-8')
record = pd.DataFrame(columns=["nickname", "comment"])

while count < 400:
    encText = get_param(json.dumps(data), var3, i)

    try:
        with requests.post(url, data={"params": encText, "encSecKey": encSecKey}) as resp:
            text = resp.json()
            for comment in text["data"]["comments"]:
                record.loc[count, "nickname"] = comment["user"]["nickname"]
                record.loc[count, "comment"] = comment["content"]
                print(comment["user"]["nickname"].strip(), ":", comment["content"].strip(), '\n', file=file)
                count += 1
    except KeyError as e:
        print("参数错误:", e)
        break
    else:
        data["offset"] = count
        print(f"{count}条评论爬取成功")
        data["pageNo"] = data["pageNo"] + 1
file.close()
print("over!!!")

20条评论爬取成功
40条评论爬取成功
60条评论爬取成功
80条评论爬取成功
100条评论爬取成功
120条评论爬取成功
140条评论爬取成功
160条评论爬取成功
180条评论爬取成功
200条评论爬取成功
参数错误: 'data'
over!!!


In [6]:
with pd.ExcelWriter("./comments.xlsx") as file:
    record.to_excel(file, index=False)